# ANALYZE EXPERIMENTS

In [9]:
import smarte.constants as cn
import smarte as smt

from dask.distributed import Client
import dask.dataframe as dd
import dask
import os
import pandas as pd
from io import StringIO
import zipfile

In [2]:
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:34947 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.21 GB


# Helpers

In [27]:
def extractZipped(file_name="experiments.zip"):
    # Extract the compressed data
    archive_path = os.path.join(cn.EXPERIMENT_DIR, "experiments.zip")
    dfs = []
    with zipfile.ZipFile(archive_path) as myzip:
        for ffile in myzip.namelist():
            with myzip.open(ffile) as myfile:
                byte_lines = (myfile.readlines())
                lines = [l.decode() for l in byte_lines]
                lines = "\n".join(lines)
                df = pd.read_csv(StringIO(lines))
                dfs.append(df)
    return pd.concat(dfs)

# Data Overview

In [29]:
DF = extractZipped()
DF.head()

,biomodel_num,num_species,num_reaction,num_parameter,columns_deleted,max_fev,method,noise_mag,num_latincube,range_min_frac,range_max_frac,ts_instance,median_err,max_err,min_err,avg_time,cnt,tot_time,status
0,1,12.0,17.0,35.0,0,10000,differential_evolution,0.1,1,0.5,2.0,1,-0.006678,0.998357,-0.006678,0.000226,1087.0,0.245964,Success!
1,1,12.0,17.0,35.0,0,10000,differential_evolution,0.1,1,0.5,2.0,2,0.268319,0.999995,0.056120,0.000233,1087.0,0.253108,Success!
2,1,12.0,17.0,35.0,0,10000,differential_evolution,0.1,1,0.5,2.0,3,0.176863,0.999158,0.032290,0.000260,1087.0,0.282281,Success!
3,1,12.0,17.0,35.0,0,10000,differential_evolution,0.1,1,0.5,2.0,4,-0.101451,-0.999998,0.041311,0.000267,1087.0,0.290280,Success!
4,1,12.0,17.0,35.0,0,10000,differential_evolution,0.1,1,0.5,2.0,5,0.345762,-0.999840,-0.067118,0.000264,1087.0,0.286798,Success!


# Create the dataframe of experiments

In [3]:
if Fale:
    DF = dd.read_csv(os.path.join(cn.EXPERIMENT_DIR, "*.csv"))

,biomodel_num,num_species,num_reaction,num_parameter,columns_deleted,max_fev,method,noise_mag,num_latincube,range_min_frac,range_max_frac,ts_instance,median_err,max_err,min_err,avg_time,cnt,tot_time,status
npartitions=16,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,int64,int64,object,float64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
if False:
    path = os.path.join(cn.EXPERIMENT_DIR, "workunits.txt")
    with open(path, "r") as fd:
        workunit_strs = fd.readlines()
    workunit_strs = [w if w[-1] == "l" else w[:-1] for w in workunit_strs]
    workunits = [smt.Workunit.getFromStr(w) for w in workunit_strs]

# Missing Experiments

In [6]:
set(["--".join(w.calcMultivaluedFactors()) for w in workunits])

{'biomodel_num--ts_instance'}

In [7]:
# Find what varies across workunits

In [8]:
# Combine workunits
agg_workunits = smt.ExtendedDict({k: [] for k in cn.SD_CONDITIONS})
for workunit in workunits[1:]:
    dct = workunit.removeExpansions()
    for key in dct.keys():
        # Handle bug where get [["differential_evolution"]]
        if isinstance(dct[key], str):
            values = set(agg_workunits[key]).union(dct[key])
        elif isinstance(dct[key], list):
            values = set(agg_workunits[key]).union(dct[key][0])
        agg_workunits[key] = list(values)
    print(agg_workunits)
if False:
    # Remove those known to vary
    for factor in [cn.SD_BIOMODEL_NUM, cn.SD_TS_INSTANCE]:
        del agg_workunits[factor]
    str(agg_workunits)

TypeError: 'int' object is not iterable

In [55]:
count_df = pd.DataFrame(DF.groupby([cn.SD_NOISE_MAG, cn.SD_NUM_LATINCUBE, cn.SD_METHOD, cn.SD_MAX_FEV]).count()["biomodel_num"])
count_df = count_df.rename(columns={"biomodel_num": "num_experiments"})
pd.DataFrame(count_df)

num_experiments
noise_mag num_latincube method                 max_fev                 
0.1       1             differential_evolution 1000                3425
                                               10000               3425
                        leastsq                1000                3425
                                               10000               3425
          2             differential_evolution 1000                3425
                                               10000               2316
                        leastsq                1000                3425
                                               10000               1489
          4             differential_evolution 1000                3425
                                               10000                725
                        leastsq                1000                3425
                                               10000               1485
          8             differential_evolution 1000                1491
                                               10000                725
                        leastsq                1000                3425
                                               10000               1485
0.2       1             differential_evolution 1000                3425
                                               10000               4086
                        leastsq                1000                3425
                                               10000               3425
          2             differential_evolution 1000                3425
                                               10000               1486
                        leastsq                1000                3425
                                               10000               2637
          4             differential_evolution 1000                3425
                                               10000                725
                        leastsq                1000                3425
                                               10000               1486
          8             differential_evolution 1000                3425
                                               10000                725
                        leastsq                1000                3425
                                               10000               1485

In [ ]:
dff = DF[[cn.SD_NOISE_MAG, cn.SD_RANGE_INITIAL_FRAC, cn.SD_METHOD]]
len(dff)

In [26]:
len(dff)

83996